### Experiments
**Datasets**: Rao2014 GSM1551550_HIC001.hic <br>
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1551550 <br>
**Chromosome**: Local (Eg: chrom4 vs chrom4, chrom5 vs chrom5) <br>
**Normalization**: KR <br>
**Resolution**: 1Mb (1 mega base pair) <br>

* Visualize the .hic file by Juicebox, and Compare with the Figure in Lieberman-2009 Supplement.
* Calculate the Pearson matrix and eigenvector1 by the Juicer tools
* Calculate the O/E matrix, Pearson matrix and eigenvector1 by the Straw and Sklearn

### Results
* Using **Juicebox** to visualize the `HIC001.hic`, the plaid patterns looks the same as the figure in the experiments of Aiden-Lieberman 2009
* If the entries of the origin O/E matrix are Zero -> The corresponding entries in the Pearson matrix will be NaN. (Check out the files in `data/outputs/juicer`)
* The results produced by **Juicer_tools** are almost the same as producing with **Straw + Sklearn**. (In the eigenvector1, if there exist missing values, the Juicer tools will produce NaN while the Sklearn will produce Zero)
  * The NaN or Zero entries all exist in the same line of the output `.txt` files.  
* The eigenvalues are all positive, and the explained variance:
  * In the chrom1 and chrom4, first 3 PCs seems ok (PC1: 0.7xx)
  * In the chrom5 (0.56, 0.39, 0.015)
  * The difference of PC1 between Juicer_tools and Straw + Sklearn is not obvious, it is worth noting that the sign of the entries of PC1 **flipped** between Juicer_tools and Straw + Sklearn in **Chrom5** (Not all Chromosome fit this situation).
  * If the size of using dataset is small, the explained variance will decrease.(test.hic) 

In [2]:
%%bash
rm -rf /home/jordan990301/PCA_Experiments/data/outputs/juicer
mkdir /home/jordan990301/PCA_Experiments/data/outputs/juicer

HIC_PATH="/home/jordan990301/PCA_Experiments/data/Rao2014/samples/GSM1551550_HIC001.hic"
OUTPUT_PATH="/home/jordan990301/PCA_Experiments/data/outputs/juicer"
JUICER_TOOLS_PATH="/home/jordan990301/PCA_Experiments/juicer/juicer_tools.jar"

java -jar $JUICER_TOOLS_PATH pearsons KR $HIC_PATH 1 BP 1000000 $OUTPUT_PATH/pearson_matrix.txt -p
java -jar $JUICER_TOOLS_PATH eigenvector KR $HIC_PATH 1 BP 1000000 $OUTPUT_PATH/eigenvector.txt -p

WARN [2023-11-08T17:54:27,496]  [Globals.java:138] [main]  Development mode is enabled
Reading file: /home/jordan990301/PCA_Experiments/data/Rao2014/samples/GSM1551550_HIC001.hic
WARN [2023-11-08T17:54:28,510]  [Globals.java:138] [main]  Development mode is enabled


In [3]:
import sys
import numpy as np
import pandas as pd
import hicstraw
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import pyplot as plt
from matplotlib import gridspec
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
hic_path = "/home/jordan990301/PCA_Experiments/data/Rao2014/samples/GSM1551550_HIC001.hic"
output_path = "/home/jordan990301/PCA_Experiments/data/outputs/juicer"
chrom_name_x = "1"
chrom_name_y = "1"
chrom_type = "BP"
norm = "KR"
resolution = 1000000

In [5]:
hic = hicstraw.HiCFile(hic_path)

In [6]:
print(hic.getGenomeID())
print(hic.getResolutions())

hg19
[2500000, 1000000, 500000, 250000, 100000, 50000, 25000, 10000, 5000]


In [7]:
selected_chromosome_size = None

for chrom in hic.getChromosomes():
    print(chrom.name, chrom.length)
    if (chrom.name == chrom_name_x):
        selected_chromosome_size = int(chrom.length)

print(f"\n{selected_chromosome_size}")

All 3098789
1 249250621
2 243199373
3 198022430
4 191154276
5 180915260
6 171115067
7 159138663
8 146364022
9 141213431
10 135534747
11 135006516
12 133851895
13 115169878
14 107349540
15 102531392
16 90354753
17 81195210
18 78077248
19 59128983
20 63025520
21 48129895
22 51304566
X 155270560
Y 59373566
MT 16569

249250621


In [8]:
matrix_oe = hic.getMatrixZoomData(chrom_name_x, chrom_name_y, "oe", norm, chrom_type, resolution)
numpy_matrix_oe = matrix_oe.getRecordsAsMatrix(0, selected_chromosome_size, 0, selected_chromosome_size)

In [9]:
# oe_matrix_df = pd.DataFrame(numpy_matrix_oe)
# display(oe_matrix_df)

# np.savetxt(f'{output_path}/oe_matrix.txt', oe_matrix_df.values, fmt='%1.4e')

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,1.866086,1.473780,0.522184,0.499840,0.288058,0.419308,0.898901,0.588248,0.830080,0.764695,...,0.224341,0.292870,0.626094,0.662700,0.894858,0.630684,1.289456,0.799445,1.901437,3.566629
1,1.473780,1.545669,0.780814,1.098850,0.698886,0.843478,1.525545,0.905102,0.931570,1.237671,...,0.181945,0.256742,0.244929,0.548277,0.329073,0.464630,1.279799,0.898430,0.587639,0.518713
2,0.522184,0.780814,1.272226,1.572296,2.161446,2.903659,1.709219,1.998500,0.608008,0.529149,...,0.469217,0.453269,0.420660,0.263664,0.634491,0.508970,0.609830,0.635267,0.716875,0.186115
3,0.499840,1.098850,1.572296,1.208327,1.113256,2.226933,1.761914,1.582344,0.561516,0.448468,...,0.310039,0.839651,0.533459,0.587303,0.924089,0.842408,0.857589,0.751051,0.624688,0.539633
4,0.288058,0.698886,2.161446,1.113256,1.143983,1.815590,0.760112,1.514706,0.341567,0.212668,...,1.246444,1.040834,0.809236,0.794585,1.222127,1.514367,0.636639,0.784589,0.939637,0.382578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.630684,0.464630,0.508970,0.842408,1.514367,1.641419,1.267794,1.680717,0.684394,0.687388,...,1.231908,1.291951,1.209224,0.901555,1.112232,1.046352,1.643811,1.090796,0.798989,0.652613
246,1.289456,1.279799,0.609830,0.857589,0.636639,0.872614,1.628995,1.178725,1.422031,1.011166,...,1.009663,1.201164,0.981190,1.200778,1.429020,1.643811,1.178887,0.921359,0.550331,0.797736
247,0.799445,0.898430,0.635267,0.751051,0.784589,1.044210,1.024072,0.816397,0.629533,0.586616,...,2.096076,1.734301,1.528470,1.347306,1.241600,1.090796,0.921359,1.221308,0.934723,1.338616
248,1.901437,0.587639,0.716875,0.624688,0.939637,0.475875,0.604751,0.888152,0.800450,0.303439,...,2.001054,1.758616,1.452898,0.989254,0.945536,0.798989,0.550331,0.934723,1.153779,2.202421


In [10]:
# corr_matrix_df = oe_matrix_df.corr(method='pearson')
# display(corr_matrix_df)

# np.savetxt(f'{output_path}/corr_matrix.txt', corr_matrix_df.values, fmt='%1.4e')

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,1.000000,0.655238,0.397795,0.428727,0.201170,0.193704,0.596437,0.333002,0.657187,0.635905,...,-0.135845,-0.158142,-0.163068,0.107601,0.126856,0.161430,0.262935,0.118259,-0.052065,0.265317
1,0.655238,1.000000,0.525914,0.479068,0.156343,0.159742,0.799490,0.463051,0.867195,0.898821,...,-0.350892,-0.358813,-0.348486,-0.065517,0.076147,0.123033,0.266054,-0.063602,-0.239308,0.165231
2,0.397795,0.525914,1.000000,0.934680,0.827378,0.806603,0.601443,0.708325,0.384344,0.335560,...,0.038254,-0.064669,-0.037326,0.002795,0.120971,0.323632,0.170611,0.226819,-0.004038,0.005862
3,0.428727,0.479068,0.934680,1.000000,0.851143,0.848708,0.621081,0.750194,0.379235,0.302806,...,0.080387,-0.012130,0.023424,0.067214,0.165280,0.364011,0.209976,0.258725,0.037875,0.042824
4,0.201170,0.156343,0.827378,0.851143,1.000000,0.956281,0.282321,0.570479,0.052067,-0.024065,...,0.343478,0.198102,0.239200,0.125925,0.169635,0.367646,0.116283,0.414060,0.229185,0.017004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.161430,0.123033,0.323632,0.364011,0.367646,0.416971,0.298802,0.433029,0.250367,0.139352,...,0.548658,0.604927,0.666615,0.775085,0.863622,1.000000,0.763442,0.689840,0.623381,0.499951
246,0.262935,0.266054,0.170611,0.209976,0.116283,0.135705,0.363532,0.232380,0.434310,0.354116,...,0.308858,0.438305,0.475725,0.790103,0.805529,0.763442,1.000000,0.639177,0.526767,0.637637
247,0.118259,-0.063602,0.226819,0.258725,0.414060,0.373528,-0.024928,0.048469,-0.013458,-0.059642,...,0.779284,0.782175,0.787202,0.783496,0.725811,0.689840,0.639177,1.000000,0.862474,0.572765
248,-0.052065,-0.239308,-0.004038,0.037875,0.229185,0.221575,-0.181023,-0.047946,-0.171815,-0.212326,...,0.819180,0.849699,0.850718,0.754966,0.683696,0.623381,0.526767,0.862474,1.000000,0.624370


In [12]:
### Calculate from juicer_tools
pearson_df = pd.read_csv(f"{output_path}/pearson_matrix.txt", header=None, sep=" ")
pearson_df.pop(pearson_df.columns[-1])
pearson_df

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,1.000000,0.460813,0.218598,0.220897,-0.101588,-0.191491,-0.038220,-0.272484,0.097096,0.266391,...,0.005520,-0.094553,-0.117888,-0.121898,-0.393873,-0.514383,-0.327284,-0.131620,0.067699,0.732693
1,0.460813,1.000000,0.360592,0.278837,-0.122981,-0.161327,0.635176,0.137202,0.747072,0.854184,...,-0.541381,-0.618487,-0.636442,-0.482324,-0.429188,-0.436422,-0.171906,-0.427550,-0.478513,0.067443
2,0.218598,0.360592,1.000000,0.908033,0.738623,0.680411,0.322208,0.484798,0.062958,0.114297,...,-0.148253,-0.332119,-0.325540,-0.532326,-0.492620,-0.243353,-0.433579,-0.149755,-0.272267,-0.050227
3,0.220897,0.278837,0.908033,1.000000,0.770044,0.726955,0.290700,0.504614,0.004579,0.029250,...,-0.106951,-0.285169,-0.268886,-0.498153,-0.499138,-0.252450,-0.434841,-0.153389,-0.253854,-0.026139
4,-0.101588,-0.122981,0.738623,0.770044,1.000000,0.938643,-0.022359,0.419086,-0.279205,-0.304491,...,0.210982,0.027274,0.056619,-0.263848,-0.242761,0.030243,-0.331471,0.171295,-0.000077,-0.189034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,-0.514383,-0.436422,-0.243353,-0.252450,0.030243,0.095751,-0.225313,-0.004486,-0.162936,-0.267186,...,0.332482,0.421527,0.490488,0.566613,0.745491,1.000000,0.597542,0.500957,0.315467,-0.249431
246,-0.327284,-0.171906,-0.433579,-0.434841,-0.331471,-0.320481,-0.071102,-0.285005,0.167644,0.092785,...,0.024328,0.193955,0.220045,0.609267,0.672523,0.597542,1.000000,0.456776,0.217449,-0.097925
247,-0.131620,-0.427550,-0.149755,-0.153389,0.171295,0.091556,-0.562523,-0.452999,-0.397956,-0.375970,...,0.705640,0.714964,0.717006,0.677207,0.581965,0.500957,0.456776,1.000000,0.773283,0.208319
248,0.067699,-0.478513,-0.272267,-0.253854,-0.000077,-0.052443,-0.733828,-0.542372,-0.570166,-0.491263,...,0.801464,0.822207,0.816020,0.664174,0.455861,0.315467,0.217449,0.773283,1.000000,0.535396


In [13]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
imp.fit(pearson_df)
pearson_df = imp.transform(pearson_df)

In [14]:
# corr_matrix_df doesn't exist NaN
print(np.isnan(pearson_df).any())
print(len(pearson_df[0]))
print(len(pearson_df[1]))

False
250
250


In [15]:
from sklearn.decomposition import PCA
pca = PCA(n_components=len(pearson_df[0]))
pca.fit(pearson_df)

PCA(n_components=250)

In [16]:
# print(pca.explained_variance_) # eigenvalues
print(pca.explained_variance_ratio_[0]) # Percentage of variance explained by each of the selected components.
print(pca.explained_variance_ratio_[1]) # Percentage of variance explained by each of the selected components.
print(pca.explained_variance_ratio_[2]) # Percentage of variance explained by each of the selected components.
# print(pca.components_[0])

0.7678593821808731
0.1266502519099186
0.05762409381457042


In [17]:
first_eigenvector = pca.components_[0]

for i in first_eigenvector:
    print(i)

0.05140789103013033
0.08643700539471177
0.03738655913718772
0.03275735188045908
-0.011340982636768148
-0.009183394087408467
0.07606576795377558
0.04150924094105475
0.07979863911557905
0.08560493273045289
0.08644297716886547
0.08912858027284268
0.059587954693921995
0.01333293517392839
0.01589006764705809
0.059542412599110146
0.05944117518771894
0.04627552874598923
0.03543702011385077
0.07844644563207
0.07139975136443462
0.08954289495095522
0.08407397329971679
0.08966871413526165
0.08025417793117999
0.07706752637930014
0.08416387044975883
0.08044615009677547
0.07912304809951189
0.07754794328956306
0.039126075381042125
0.0852240869283254
0.08528242985594045
0.0860188147177862
0.029028055540361888
0.07422403192345575
0.08796835402255923
0.05693763100421284
0.0832319031435062
0.08230943124018095
0.0789396024756899
0.07289228022080708
0.06558907999638894
0.08176140701402926
0.07894822251114936
0.07761223266638052
0.07516085056799081
0.01637945831062927
-0.0461588275740742
-0.0644009068491526

In [22]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.read_table(f"{output_path}/pearson_matrix.txt", header=None, sep=" ")